In [1]:
import pandas as pd
import requests
import time
from datetime import datetime

In [2]:
df = pd.read_csv("character_level_2024-10-17.csv", encoding='utf-8') 

In [4]:
print(df.isnull().sum())

ocid                                0
ranking                             0
character_name                      0
world_name                          0
character_gender                34978
character_class                 34978
character_class_level           34978
character_level                 34978
character_exp                   34978
character_exp_rate              34978
character_guild_name           171482
character_date_create           97955
access_flag                     34978
liberation_quest_clear_flag     34978
popularity                      34978
arcane_sum                          0
authentic_sum                       0
set_absorlab_count                  0
set_lubatiss_count                  0
set_arcane_count                    0
set_eternal_count                   0
dojang_best_floor               34979
union_level                     35013
union_artifact_level            48686
union_artifact_exp              48686
union_artifact_point            48686
dtype: int64

In [5]:
print(df.isnull().mean().sort_values(ascending=False))

character_guild_name           0.292291
character_date_create          0.166964
union_artifact_point           0.082985
union_artifact_exp             0.082985
union_artifact_level           0.082985
union_level                    0.059680
dojang_best_floor              0.059622
popularity                     0.059620
access_flag                    0.059620
liberation_quest_clear_flag    0.059620
character_exp_rate             0.059620
character_exp                  0.059620
character_level                0.059620
character_class_level          0.059620
character_class                0.059620
character_gender               0.059620
ranking                        0.000000
arcane_sum                     0.000000
authentic_sum                  0.000000
set_absorlab_count             0.000000
set_lubatiss_count             0.000000
set_arcane_count               0.000000
set_eternal_count              0.000000
world_name                     0.000000
character_name                 0.000000


In [6]:
print(f"전체 행 수: {len(df)}")

전체 행 수: 586682


In [3]:
# character_level이 270인 행만 남기기
df_270 = df[df['character_level'] >= 270]
# 결과 확인
print(f"남은 행 수: {len(df_270)}")
df_270.head()


남은 행 수: 311211


,ocid,ranking,character_name,world_name,character_gender,character_class,character_class_level,character_level,character_exp,character_exp_rate,...,authentic_sum,set_absorlab_count,set_lubatiss_count,set_arcane_count,set_eternal_count,dojang_best_floor,union_level,union_artifact_level,union_artifact_exp,union_artifact_point
0,4909d2b63165a45a284d0dae4478885d,3543,꾸망,스카니아,여,"아크메이지(썬,콜)",6.0,290.0,1.196775e+14,40.664,...,70,0,0,5,4,82.0,8974.0,52.0,9237.0,16580.0
1,356360924b8c044e58eaaf862766c8daefe8d04e6d233b...,8190,아란마하콤보,엘리시움,남,아란,6.0,289.0,1.157394e+13,7.944,...,66,5,2,1,0,60.0,9745.0,47.0,25735.0,16400.0
2,f7aaaa3ae22c47720c70cac37d7574e0,6406,에피파니,스카니아,남,듀얼블레이더,6.0,289.0,1.084834e+14,74.459,...,63,0,0,0,1,63.0,8987.0,49.0,13650.0,16800.0
3,987bfae423b5a6d217bc75116042200c,1144,날개,크로아,남,나이트로드,6.0,291.0,1.757344e+14,54.283,...,58,0,4,5,1,68.0,9243.0,49.0,11385.0,16800.0
4,dc3038c85c41146a9537a3f7c11369a7,6192,사클,스카니아,여,윈드브레이커,6.0,289.0,1.232270e+14,84.578,...,66,0,4,5,1,89.0,9164.0,51.0,39552.0,17300.0


In [7]:
print(df_270.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           49553
character_date_create          23545
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        3
union_artifact_level            3948
union_artifact_exp              3948
union_artifact_point            3948
dtype: int64


In [10]:
import pandas as pd

# 날짜 리스트
date_list = [
    "2024-10-17", "2024-10-24", "2024-10-31",
    "2024-11-07", "2024-11-14", "2024-11-21",
    "2024-11-28", "2024-12-05", "2024-12-12", "2024-12-19"
]

# 1. 기준일 ocid 확보
df_base = pd.read_csv("character_level_2024-10-17.csv")
ocid_1017 = set(df_base["ocid"])
df_merged = None

# 2. 주차별 파일 병합
for date in date_list:
    tag = date[-5:].replace("-", "")  # 예: 1017
    file_path = f"character_level_{date}.csv"
    df = pd.read_csv(file_path)

    df = df[["ocid", "access_flag", "character_exp"]].copy()
    df.rename(columns={
        "access_flag": f"access_{tag}",
        "character_exp": f"exp_{tag}"
    }, inplace=True)

    if df_merged is None:
        df_merged = df
    else:
        df_merged = pd.merge(df_merged, df, on="ocid", how="outer")

# 3. 기준 ocid만 필터링
df_merged = df_merged[df_merged["ocid"].isin(ocid_1017)].copy()

# 4. event 계산 함수
def define_event(row):
    access_cols = [f"access_{d[-5:].replace('-', '')}" for d in date_list]
    exp_cols = [f"exp_{d[-5:].replace('-', '')}" for d in date_list]

    access_vals = row[access_cols].tolist()
    exp_vals = row[exp_cols].tolist()

    for i in range(len(access_vals) - 1):
        if access_vals[i] == False and access_vals[i+1] == False:
            if all(v is False for v in access_vals[i+2:]):
                return 1

    if all(x == exp_vals[0] for x in exp_vals):
        return 1

    return 0

# 5. duration 계산 함수
def calculate_duration(row):
    exp_cols = [f"exp_{d[-5:].replace('-', '')}" for d in date_list]
    exps = row[exp_cols].tolist()

    duration = 1
    for i in range(1, len(exps)):
        if exps[i] != exps[i-1]:
            duration += 1
        else:
            break
    return duration

# 6. 적용
df_merged["event"] = df_merged.apply(define_event, axis=1)
df_merged["duration"] = df_merged.apply(calculate_duration, axis=1)

# 7. 결과 저장
result_df = df_merged[["ocid", "event", "duration"]]
result_df.to_csv("survival_result.csv", index=False)

# 🔍 추가: 10/17 이후 단 한 번도 등장하지 않은 유저 찾기
ocid_later = set()
for date in date_list[1:]:  # 10/24~12/19
    df = pd.read_csv(f"character_level_{date}.csv")
    ocid_later.update(df["ocid"].unique())

only_1017_ocids = ocid_1017 - ocid_later
print(f"🔍 10/17 이후 완전히 사라진 유저 수: {len(only_1017_ocids)}")

# 선택적으로 따로 저장
pd.DataFrame({"ocid": list(only_1017_ocids)}).to_csv("only_1017_ocids.csv", index=False)


🔍 10/17 이후 완전히 사라진 유저 수: 267


In [8]:
df_270.to_csv("character_level_2024-10-17.csv", index=False)

In [11]:
# 10/17에 있었는데 이후 파일 전부에서 안 나오는 OCID 목록 보기
for ocid in list(only_1017_ocids)[:5]:  # 예시로 5개만
    print(f"🔍 OCID {ocid} 존재 여부 추적:")
    for date in date_list[1:]:
        df_temp = pd.read_csv(f"character_level_{date}.csv")
        exists = ocid in df_temp["ocid"].values
        print(f"  {date}: {'✅ 있음' if exists else '❌ 없음'}")


🔍 OCID 11a0476903f4c67e798b027c683beb5b 존재 여부 추적:
  2024-10-24: ❌ 없음
  2024-10-31: ❌ 없음
  2024-11-07: ❌ 없음
  2024-11-14: ❌ 없음
  2024-11-21: ❌ 없음
  2024-11-28: ❌ 없음
  2024-12-05: ❌ 없음
  2024-12-12: ❌ 없음
  2024-12-19: ❌ 없음
🔍 OCID 0a8b7ed01025d8898e3a96447b0b1bc9 존재 여부 추적:
  2024-10-24: ❌ 없음
  2024-10-31: ❌ 없음
  2024-11-07: ❌ 없음
  2024-11-14: ❌ 없음
  2024-11-21: ❌ 없음
  2024-11-28: ❌ 없음
  2024-12-05: ❌ 없음
  2024-12-12: ❌ 없음
  2024-12-19: ❌ 없음
🔍 OCID d48f17257465d68f51bf25bd301ea393 존재 여부 추적:
  2024-10-24: ❌ 없음
  2024-10-31: ❌ 없음
  2024-11-07: ❌ 없음
  2024-11-14: ❌ 없음
  2024-11-21: ❌ 없음
  2024-11-28: ❌ 없음
  2024-12-05: ❌ 없음
  2024-12-12: ❌ 없음
  2024-12-19: ❌ 없음
🔍 OCID 09fe715758f3f0a4298876d30db06a78 존재 여부 추적:
  2024-10-24: ❌ 없음
  2024-10-31: ❌ 없음
  2024-11-07: ❌ 없음
  2024-11-14: ❌ 없음
  2024-11-21: ❌ 없음
  2024-11-28: ❌ 없음
  2024-12-05: ❌ 없음
  2024-12-12: ❌ 없음
  2024-12-19: ❌ 없음
🔍 OCID ac4857b3f0caf5a20cab1ead94a9c651efe8d04e6d233bd35cf2fabdeb93fb0d 존재 여부 추적:
  2024-10-24: ❌ 없음
  2024-10-31: ❌

In [12]:
import pandas as pd

# 1. 파일 불러오기
df_original = pd.read_csv("character_level_2024-10-17.csv")
df_survival = pd.read_csv("survival_result.csv")

# 2. ocid 기준으로 병합 (원본 기준 유지)
df_combined = pd.merge(df_original, df_survival, on="ocid", how="left")

# 3. 결과 저장
df_combined.to_csv("character_level_2024-10-17_with_survival.csv", index=False)


In [13]:
df=pd.read_csv("character_level_2024-10-17_with_survival.csv", encoding='utf-8') 

In [14]:
print(df.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           49553
character_date_create          23545
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        3
union_artifact_level            3948
union_artifact_exp              3948
union_artifact_point            3948
event                              0
d

In [15]:
# 10/17 이후 완전히 사라진 ocid 목록
df_survival = pd.read_csv("character_level_2024-10-17_with_survival.csv")
only_1017 = pd.read_csv("only_1017_ocids.csv")

# 제외 대상 플래그 지정
df_survival["drop_flag"] = df_survival["ocid"].isin(only_1017["ocid"])

# 제외하고 새로운 분석용 데이터셋 생성
df_survival_clean = df_survival[~df_survival["drop_flag"]].copy()
df_survival_clean.drop(columns=["drop_flag"], inplace=True)

# 저장
df_survival_clean.to_csv("survival_result_cleaned.csv", index=False)

In [16]:
df2=pd.read_csv("survival_result_cleaned.csv", encoding='utf-8') 

In [28]:
print(df3.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           49510
character_date_create          23541
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        3
union_artifact_level            3947
union_artifact_exp              3947
union_artifact_point            3947
event                              0
d

In [25]:
print(f"전체 행 수: {len(df)}")
print(f"전체 행 수: {len(df2)}")
print(f"전체 행 수: {len(df3)}")

전체 행 수: 313257
전체 행 수: 312990
전체 행 수: 312990


In [23]:
import pandas as pd
import os

# 날짜 리스트
date_list = [
    "2024-10-17", "2024-10-24", "2024-10-31",
    "2024-11-07", "2024-11-14", "2024-11-21",
    "2024-11-28", "2024-12-05", "2024-12-12", "2024-12-19"
]

# 기본 베이스: survival_result_cleaned
df_base = pd.read_csv("survival_result_cleaned.csv")

# 날짜별 경험치, access_flag 붙이기
for date in date_list:
    tag = date[-5:].replace("-", "")  # 예: "10-17" → "1017"
    file_path = f"character_level_{date}.csv"

    if os.path.exists(file_path):
        df_temp = pd.read_csv(file_path, usecols=["ocid", "character_exp", "access_flag"])
        
        # 🔧 ocid 기준 중복 제거 (첫 번째 행만 사용)
        df_temp = df_temp.drop_duplicates(subset="ocid")

        # 컬럼 이름 변경
        df_temp.rename(columns={
            "character_exp": f"exp_{tag}",
            "access_flag": f"access_{tag}"
        }, inplace=True)

        # 병합
        df_base = pd.merge(df_base, df_temp, on="ocid", how="left")
    else:
        # 파일 없으면 빈 칼럼 추가
        df_base[f"exp_{tag}"] = None
        df_base[f"access_{tag}"] = None

# 저장
df_base.to_csv("survival_result_with_exp_access.csv", index=False)


In [27]:
df3=pd.read_csv("survival_result_with_exp_access.csv", encoding='utf-8',low_memory=False) 

In [26]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312990 entries, 0 to 312989
Data columns (total 48 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ocid                         312990 non-null  object 
 1   ranking                      312990 non-null  int64  
 2   character_name               312990 non-null  object 
 3   world_name                   312990 non-null  object 
 4   character_gender             312990 non-null  object 
 5   character_class              312990 non-null  object 
 6   character_class_level        312990 non-null  float64
 7   character_level              312990 non-null  float64
 8   character_exp                312990 non-null  float64
 9   character_exp_rate           312990 non-null  float64
 10  character_guild_name         263480 non-null  object 
 11  character_date_create        289449 non-null  object 
 12  access_flag                  312990 non-null  bool   
 13 

In [34]:
# 경험치 중간에 하나라도 NaN이 있으면 제거

exp_cols = [col for col in df.columns if col.startswith("exp_")]

# 3. 경험치 컬럼들 전부 float으로 변환 (숫자로 인식되지 않은 경우 대비)
df[exp_cols] = df[exp_cols].apply(pd.to_numeric, errors='coerce')
df_cleaned = df3.dropna(subset=exp_cols)

In [35]:
print(df_cleaned.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           49510
character_date_create          23541
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        3
union_artifact_level            3947
union_artifact_exp              3947
union_artifact_point            3947
event                              0
d

In [63]:
import pandas as pd
import os

# 날짜 리스트
date_list = [
    "2024-10-17", "2024-10-24", "2024-10-31",
    "2024-11-07", "2024-11-14", "2024-11-21",
    "2024-11-28", "2024-12-05", "2024-12-12", "2024-12-19"
]

#  10월 17일 기준 데이터 불러오기
df_base = pd.read_csv("character_level_2024-10-17.csv")
df_base = df_base.drop_duplicates(subset="ocid")

# 나머지 주차의 exp, access_flag 병합
for date in date_list: 
    tag = date[-5:].replace("-", "")  
     # 파일명 설정 (10월 17일만 예외 처리)
    if date == "2024-10-17":
        file_name = f"character_level_{date}.csv"
    else:
        file_name = f"character_level_{date}_merged.csv"
    
    if os.path.exists(file_name):
        df_temp = pd.read_csv(file_name, usecols=["ocid", "character_exp", "access_flag"])
        df_temp = df_temp.drop_duplicates(subset="ocid")

        df_temp.rename(columns={
            "character_exp": f"exp_{tag}",
            "access_flag": f"access_{tag}"
        }, inplace=True)

        df_base = pd.merge(df_base, df_temp, on="ocid", how="left")
    else:
        df_base[f"exp_{tag}"] = None
        df_base[f"access_{tag}"] = None

# 저장
df_base.to_csv("merged_character_with_exp_access.csv", index=False)

# 확인용 출력
print(f"최종 병합 완료! 총 행 수: {len(df_base)}")


최종 병합 완료! 총 행 수: 311210


In [64]:
df=pd.read_csv("merged_character_with_exp_access.csv", encoding='utf-8',low_memory=False) 

In [65]:
print(df.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           49553
character_date_create          23545
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        3
union_artifact_level            3948
union_artifact_exp              3948
union_artifact_point            3948
exp_1017                           0
a

In [11]:
# 경험치 및 접속여부 컬럼 추출
exp_cols = [col for col in df.columns if col.startswith("exp_")]
access_cols = [col for col in df.columns if col.startswith("access_")]

# 결측치 제거 (exp 또는 access 중 하나라도 NaN이면 제거)
df_cleaned = df.dropna(subset=exp_cols + access_cols)

# 결과 저장
df_cleaned.to_csv("merged_exp_access_no_nan.csv", index=False)

# 결과 요약 출력
print(f" 결측 제거 전: {len(df)}명")
print(f" 결측 제거 후: {len(df_cleaned)}명 (완전한 주차별 exp/access 기록 보유)")


 결측 제거 전: 311210명
 결측 제거 후: 310033명 (완전한 주차별 exp/access 기록 보유)


In [12]:
# 제거된 유저만 추출 (즉, 널값이 하나라도 있었던 유저)
df_dropped = df[~df["ocid"].isin(df_cleaned["ocid"])]

# 확인용 출력
print(f"총 유저 수: {len(df)}")
print(f"널값 있는 유저 수: {len(df_dropped)}")

# 파일로 저장
df_dropped.to_csv("dropped_users_with_nan.csv", index=False)


총 유저 수: 311210
널값 있는 유저 수: 1177


In [14]:
import pandas as pd

# 파일 불러오기
file_path = "dropped_users_with_nan.csv"
df_dropped = pd.read_csv(file_path)

# 컬럼별 결측치 개수 요약
missing_summary = df_dropped.isnull().sum().sort_values(ascending=False)

# 결측치가 있는 주차별 컬럼만 추출 (exp_ 또는 access_ 포함된 컬럼만)
missing_exp_access = missing_summary[missing_summary.index.str.startswith(("exp_", "access_"))]

# 상위 몇 개만 확인
missing_exp_access.head(20)


access_1128    758
exp_1128       758
access_1121    680
exp_1121       680
exp_1212       669
access_1212    667
exp_1219       656
access_1219    648
exp_1205       627
access_1205    626
access_1114    470
exp_1114       470
access_1107    433
exp_1107       433
exp_1031       425
access_1031    417
exp_1024       386
access_1024    380
access_flag      0
access_1017      0
dtype: int64

In [15]:
df_dropped[df_dropped["missing_weeks"] == 0]


KeyError: 'missing_weeks'

In [44]:
df2=pd.read_csv("merged_exp_access_no_nan.csv", encoding='utf-8',low_memory=False) 

In [63]:
print(df2.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           49313
character_date_create          23523
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        3
union_artifact_level            3945
union_artifact_exp              3945
union_artifact_point            3945
exp_1017                           0
a

In [4]:
# access_1017이 True인 유저 수
num_active_start = df[df["access_1017"] == True].shape[0]
print(f" 10월 17일 기준 접속 유저 수: {num_active_start}")


 10월 17일 기준 접속 유저 수: 222939


In [6]:
df5=pd.read_csv("survival_dataset_alive_only2.csv", encoding='utf-8',low_memory=False) 

In [7]:
# 경험치 및 접속여부 컬럼 추출
exp_cols = [col for col in df.columns if col.startswith("exp_")]
access_cols = [col for col in df.columns if col.startswith("access_")]

# 결측치 제거 (exp 또는 access 중 하나라도 NaN이면 제거)
df5_cleaned = df5.dropna(subset=exp_cols + access_cols)

# 결과 저장
df5_cleaned.to_csv("merged_exp_access_no_nan2.csv", index=False)

# 결과 요약 출력
print(f" 결측 제거 전: {len(df_merged)}명")
print(f" 결측 제거 후: {len(df_cleaned)}명 (완전한 주차별 exp/access 기록 보유)")
# 제거된 유저만 추출 (즉, 널값이 하나라도 있었던 유저)
df_dropped = df[~df["ocid"].isin(df_cleaned["ocid"])]

# 확인용 출력
print(f"총 유저 수: {len(df)}")
print(f"널값 있는 유저 수: {len(df_dropped)}")

# 파일로 저장
df_dropped.to_csv("dropped_users_with_nan.csv", index=False)


ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           16829
character_date_create              0
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        0
union_artifact_level               0
union_artifact_exp                 0
union_artifact_point               0
exp_1017                           0
a

In [66]:
# 생존분석 대상: access_1017 == True
df_alive = df[df["access_1017"] == True].copy()

# 복귀/이탈 분석 대상: access_1017 == False
df_dead_or_return = df[df["access_1017"] == False].copy()

# CSV 저장
df_alive.to_csv("survival_dataset_alive_only.csv", index=False)
df_dead_or_return.to_csv("return_or_left_dataset.csv", index=False)

print(f" 생존분석 대상 저장 완료: {len(df_alive)}명")
print(f" 복귀/이탈 분석 대상 저장 완료: {len(df_dead_or_return)}명")


 생존분석 대상 저장 완료: 222939명
 복귀/이탈 분석 대상 저장 완료: 88271명


In [70]:
df2=pd.read_csv("survival_dataset_alive_only.csv", encoding='utf-8',low_memory=False) 

In [68]:
print(df2.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           16829
character_date_create              0
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        0
union_artifact_level               0
union_artifact_exp                 0
union_artifact_point               0
exp_1017                           0
a

In [72]:
# 경험치 및 접속여부 컬럼 추출
exp_cols = [col for col in df2.columns if col.startswith("exp_")]
access_cols = [col for col in df2.columns if col.startswith("access_")]

# 결측치 제거 (exp 또는 access 중 하나라도 NaN이면 제거)
df2_cleaned = df2.dropna(subset=exp_cols + access_cols)

# 결과 저장
df2_cleaned.to_csv("final_data.csv", index=False)

# 결과 요약 출력
print(f" 결측 제거 전: {len(df2)}명")
print(f" 결측 제거 후: {len(df2_cleaned)}명 (완전한 주차별 exp/access 기록 보유)")
# 제거된 유저만 추출 (즉, 널값이 하나라도 있었던 유저)
df_dropped = df2[~df2["ocid"].isin(df2_cleaned["ocid"])]

# 확인용 출력
print(f"총 유저 수: {len(df2)}")
print(f"널값 있는 유저 수: {len(df_dropped)}")

# 파일로 저장
df_dropped.to_csv("dropped_users_with_nan.csv", index=False)


 결측 제거 전: 222939명
 결측 제거 후: 222937명 (완전한 주차별 exp/access 기록 보유)
총 유저 수: 222939
널값 있는 유저 수: 2


In [93]:
df = pd.read_csv("final_data_changed_only.csv")

In [94]:
print(df.isnull().sum())

ocid                               0
exp_1010                         651
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           12803
character_date_create              0
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        0
union_artifact_level               0
union_artifact_exp                 0
union_artifact_point               0
e

In [91]:
import pandas as pd

# 1. exp 변화 여부로 생존 기간 계산
def calculate_duration(row):
    exp_cols = [col for col in row.index if col.startswith("exp_")]
    exp_vals = row[exp_cols].tolist()

    last_change = 0
    for i in reversed(range(1, len(exp_vals))):
        if exp_vals[i] != exp_vals[i - 1]:
            last_change = i
            break
    return last_change + 1  # 주차는 1부터 시작한다고 가정

def define_event(row):
    access_cols = [col for col in row.index if col.startswith("access_")]
    exp_cols = [col for col in row.index if col.startswith("exp_")]

    access_vals = [(v is True) if isinstance(v, bool) else str(v).lower() == 'true' for v in row[access_cols]]
    exp_vals = row[exp_cols].tolist()

    #  2주 연속 미접속 + 이후 전부 미접속 + 복귀 없음
    for i in range(len(access_vals) - 2):
        if not access_vals[i] and not access_vals[i + 1]:
            after = access_vals[i + 2:]
            if after and all(a is False for a in after):  # 복귀 없음
                return 1

    # 경험치가 한 번도 안 오른 경우 → 이탈
    if all(x == exp_vals[0] for x in exp_vals):
        return 1

    # exp가 오르다 마지막까지 정체 + 마지막 주차 예외
    last_change_idx = None
    for i in reversed(range(1, len(exp_vals))):
        if exp_vals[i] != exp_vals[i - 1]:
            last_change_idx = i
            break

    if last_change_idx is None:
        return 1  # 아예 변화 없음

    # 마지막 2주차 내에 변화가 있었다면 생존
    if last_change_idx >= len(exp_vals) - 2:
        return 0

    stagnant = exp_vals[last_change_idx:]
    if all(x == stagnant[0] for x in stagnant):
        return 1

    return 0

# 적용
df["duration"] = df.apply(calculate_duration, axis=1)
df["event"] = df.apply(define_event, axis=1)
df.to_csv("survival_dataset_final.csv", index=False)

print(" duration + event 계산 완료 및 저장됨: survival_dataset_final.csv")


 duration + event 계산 완료 및 저장됨: survival_dataset_final.csv


In [78]:
df=pd.read_csv("survival_dataset_final.csv", encoding='utf-8',low_memory=False) 

In [79]:
print(f"✅ 이탈율: {len(df['event']==1)}명")

✅ 이탈율: 222937명


In [80]:
print(df.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           16829
character_date_create              0
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        0
union_artifact_level               0
union_artifact_exp                 0
union_artifact_point               0
exp_1017                           0
a

In [79]:


# 경험치 + access 컬럼 추출
exp_cols = [col for col in df.columns if col.startswith("exp_")]
access_cols = [col for col in df.columns if col.startswith("access_")]

# ✅ 랜덤 샘플 추출
sample_df = df.sample(n=5, random_state=42)

# 각 샘플에 대해 exp + access + event + duration 출력
for idx, row in sample_df.iterrows():
    print(f"\n🧾 OCID: {row['ocid']}")
    print(f"Event: {row['event']} | Duration: {row['duration']}")
    print("Exp 변화:")
    for col in exp_cols:
        print(f"{col}: {row[col]}")
    print("Access Flag:")
    for col in access_cols:
        print(f"{col}: {row[col]}")



🧾 OCID: 76d39661b8f54cb3c7f530f94be08daa
Event: 0 | Duration: 10
Exp 변화:
exp_1017: 63708831960454.0
exp_1024: 68149691812422.0
exp_1031: 70482007661465.0
exp_1107: 71969677557950.0
exp_1114: 73543897944350.0
exp_1121: 74546495595648.0
exp_1128: 77446368141051.0
exp_1205: 78224734102368.0
exp_1212: 80798445473864.0
exp_1219: 87487920846475.0
Access Flag:
access_flag: True
access_1017: True
access_1024: True
access_1031: True
access_1107: True
access_1114: True
access_1121: True
access_1128: True
access_1205: True
access_1212: True
access_1219: True

🧾 OCID: b7c6b4e864a4987d06ce0299545250c0
Event: 0 | Duration: 10
Exp 변화:
exp_1017: 4257867960215.0
exp_1024: 4988683425917.0
exp_1031: 5729701341076.0
exp_1107: 6455959829196.0
exp_1114: 7194588626089.0
exp_1121: 7939309425124.0
exp_1128: 8727675365034.0
exp_1205: 9508010840391.0
exp_1212: 10305249136549.0
exp_1219: 11100423692652.0
Access Flag:
access_flag: True
access_1017: True
access_1024: True
access_1031: True
access_1107: True
access

In [80]:


# ✅ 이탈 유저 (event == 1) 샘플 100개
df_event_1 = df[df["event"] == 1].sample(n=100, random_state=42)

# ✅ 생존 유저 (event == 0) 샘플 100개
df_event_0 = df[df["event"] == 0].sample(n=100, random_state=42)

# ✅ 저장
df_event_1.to_csv("sample_event1_100.csv", index=False)
df_event_0.to_csv("sample_event0_100.csv", index=False)

print("🎯 이탈(1)과 생존(0) 각각 100개 추출 완료 및 저장됨!")


🎯 이탈(1)과 생존(0) 각각 100개 추출 완료 및 저장됨!


In [81]:
df_sample = df.sample(n=5000, random_state=42)

In [82]:
df_sample.to_csv("df_sample.csv", index=False)

In [84]:
import pandas as pd
from scipy.stats import chi2_contingency

# 1. 범주형 변수 vs event (0/1) 교차표 만들기
contingency_table = pd.crosstab(df["character_class"], df["event"])

# 2. 카이제곱 검정 실행
chi2, p, dof, expected = chi2_contingency(contingency_table)

# 3. 결과 출력
print("📊 카이제곱 독립성 검정 결과")
print(f"Chi-squared 통계량: {chi2:.4f}")
print(f"자유도 (df): {dof}")
print(f"p-value: {p:.4f}")

if p < 0.05:
    print("✔️ 직업과 이탈 여부 사이에 통계적으로 유의한 관계가 있습니다.")
else:
    print("❌ 직업과 이탈 여부는 통계적으로 유의한 차이가 없습니다.")


📊 카이제곱 독립성 검정 결과
Chi-squared 통계량: 554.5036
자유도 (df): 61
p-value: 0.0000
✔️ 직업과 이탈 여부 사이에 통계적으로 유의한 관계가 있습니다.


In [85]:
import numpy as np

# N: 총 샘플 수, k: 더 작은 차원의 수 (min(rows, columns) - 1)
N = contingency_table.values.sum()
min_dim = min(contingency_table.shape) - 1
cramers_v = np.sqrt(chi2 / (N * min_dim))
print(f"Cramér's V: {cramers_v:.4f}")


Cramér's V: 0.0500


In [87]:


# 2. 등장 수가 적은 직업 제거 (예: 30개 이상)
job_counts = df["character_class"].value_counts()
valid_jobs = job_counts[job_counts >= 30].index
df_valid = df[df["character_class"].isin(valid_jobs)]

# 3. 직업별 이탈률 계산
job_event_rate = df_valid.groupby("character_class")["event"].mean()

# 4. 이탈률 상위 10
top10_jobs = job_event_rate.sort_values(ascending=False).head(10).reset_index()
top10_jobs.columns = ["character_class", "이탈률 (상위)"]
print("\n📊 이탈률 상위 10개 직업:")
print(top10_jobs)

# 5. 이탈률 하위 10
bottom10_jobs = job_event_rate.sort_values(ascending=True).head(10).reset_index()
bottom10_jobs.columns = ["character_class", "이탈률 (하위)"]
print("\n📉 이탈률 하위 10개 직업:")
print(bottom10_jobs)

# 6. 직업군 매핑 함수
def get_class_group(cls):
    if "히어로" in cls or "팔라딘" in cls or "다크나이트" in cls or "소울마스터" in cls:
        return "전사"
    elif "아크메이지" in cls or "비숍" in cls or "플레임위자드" in cls:
        return "마법사"
    elif "나이트로드" in cls or "섀도어" in cls or "듀얼블레이더" in cls or "나이트워커" in cls:
        return "도적"
    elif "윈드브레이커" in cls or "보우마스터" in cls or "신궁" in cls or "패스파인더" in cls:
        return "궁수"



📊 이탈률 상위 10개 직업:
  character_class  이탈률 (상위)
0           나이트워커  0.170397
1             바이퍼  0.146853
2           소울마스터  0.145987
3             섀도어  0.143202
4          플레임위자드  0.142578
5              라라  0.141769
6            블래스터  0.137838
7              은월  0.137709
8      아크메이지(불,독)  0.135736
9           배틀메이지  0.135219

📉 이탈률 하위 10개 직업:
  character_class  이탈률 (하위)
0             초보자  0.056604
1             카데나  0.089660
2              제로  0.094285
3           다크나이트  0.098668
4              팬텀  0.103883
5          듀얼블레이더  0.104270
6              아크  0.105344
7           데몬어벤져  0.106939
8          데몬슬레이어  0.107004
9              카인  0.107143


In [24]:
import requests

API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
ocid = "315bde624cd11939bdbbabd4106fbe4f"
target_date = "2024-10-24"

url = f"https://open.api.nexon.com/maplestory/v1/character/basic?ocid={ocid}&date={target_date}"
headers = {
    "x-nxopen-api-key": API_KEY
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(" 캐릭터 정보:", data)
else:
    print(f"❌ 호출 실패: 상태코드 {response.status_code}")


 캐릭터 정보: {'date': '2024-10-24T00:00+09:00', 'character_name': '댜쨈', 'world_name': '스카니아', 'character_gender': '여', 'character_class': '아크메이지(불,독)', 'character_class_level': '6', 'character_level': 290, 'character_exp': 50150898506424, 'character_exp_rate': '17.040', 'character_guild_name': '미담', 'character_image': 'https://open.api.nexon.com/static/maplestory/character/look/PPPEIHDGKCCBHMJHODLCIFEOKDDENKEODIBGEGMLDANEJFPNLBHOEPICBKJNOAMDICJBFIKFCHJPFAJEOIBKGJKIIEGEPJGLNLJOBCJDFBECANGCEENMDCCDELOMPFKJNHEBAFCDLOCGCNMBDHNCDAFGNHHJDOICJNKPKJBDKLOJMJMFNKFNMEACPEECDAONFJJHGMFDGMEEBADIEPOIPFMOPGLHIBGAEBIOJPDIJNCGLLFHAJMHABOGDNIHMPOH', 'character_date_create': '2018-05-22T00:00+09:00', 'access_flag': 'true', 'liberation_quest_clear_flag': 'true'}


In [25]:
import requests

API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
ocid = "315bde624cd11939bdbbabd4106fbe4f"
target_date = "2024-10-17"

url = f"https://open.api.nexon.com/maplestory/v1/ranking/overall"
params = {
    "date": target_date,
    "ocid": ocid  #  ocid 직접 검색 가능
}

headers = {
    "x-nxopen-api-key": API_KEY
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print(" 랭킹 결과:", data)
else:
    print(f"호출 실패: 상태코드 {response.status_code}")


 랭킹 결과: {'ranking': [{'date': '2024-10-17', 'world_name': '스카니아', 'ranking': 5246, 'character_name': '댜빤', 'character_level': 290, 'character_exp': 31386070617907, 'class_name': '마법사', 'sub_class_name': '아크메이지(불,독)', 'character_popularity': 61, 'character_guildname': '미담'}]}


In [36]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# 🔐 넥슨 API 키
API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {"x-nxopen-api-key": API_KEY}

# 📥 1. ocid + date 조합 불러오기
df = pd.read_csv("df_missing_ocids_by_week.csv")  # 너가 가진 파일 이름
targets = df.to_dict("records")
def fetch_ranking_with_retry(row, max_retry=3, retry_delay=2.0):
    ocid = row["ocid"]
    date = row["date"]
    url = "https://open.api.nexon.com/maplestory/v1/ranking/overall"
    params = {"date": date, "ocid": ocid}

    attempt = 0
    while attempt < max_retry:
        try:
            res = requests.get(url, headers=headers, params=params, timeout=5)
            status_code = res.status_code

            # 429 감지되면 딜레이 후 재시도
            if status_code == 429:
                time.sleep(retry_delay)
                attempt += 1
                continue

            # 정상 응답 시
            data = res.json()

            if status_code == 200 and isinstance(data.get("ranking"), list) and len(data["ranking"]) > 0:
                entry = data["ranking"][0]
                return {
                    "ocid": ocid,
                    "date": date,
                    "ranking": entry.get("ranking"),
                    "character_name": entry.get("character_name"),
                    "world_name": entry.get("world_name"),
                    "character_gender": entry.get("character_gender"),
                    "character_class": entry.get("class_name"),
                    "sub_class_name": entry.get("sub_class_name"),
                    "character_class_level": entry.get("character_class_level"),
                    "character_level": entry.get("character_level"),
                    "character_exp": entry.get("character_exp"),
                    "character_popularity": entry.get("character_popularity"),
                    "character_guildname": entry.get("character_guildname"),
                    "exists_in_ranking": True,
                    "status_code": status_code,
                    "retry_count": attempt
                }
            else:
                return {
                    "ocid": ocid,
                    "date": date,
                    "exists_in_ranking": False,
                    "status_code": status_code,
                    "retry_count": attempt
                }

        except Exception as e:
            return {
                "ocid": ocid,
                "date": date,
                "exists_in_ranking": False,
                "status_code": None,
                "error": str(e),
                "retry_count": attempt
            }

    # 모든 시도 실패
    return {
        "ocid": ocid,
        "date": date,
        "exists_in_ranking": False,
        "status_code": 429,
        "error": "Max retries exceeded",
        "retry_count": max_retry
    }



# 🚀 3. 병렬 실행 + tqdm 진행률 표시
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_ranking, row) for row in targets]
    for future in tqdm(as_completed(futures), total=len(futures)):
        results.append(future.result())

# 💾 4. 결과 저장
df_result = pd.DataFrame(results)
df_result.to_csv("ranking_detail_parallel.csv", index=False)
print("✅ 저장 완료: ranking_detail_parallel.csv")


100%|██████████| 5104/5104 [00:18<00:00, 283.02it/s]

✅ 저장 완료: ranking_detail_parallel.csv


In [32]:
# 예시로 1건만 호출해보기 (Colab에서 해봐!)
import requests

API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {"x-nxopen-api-key": API_KEY}
ocid = "315bde624cd11939bdbbabd4106fbe4f"  # 예시
date = "2024-10-24"

url = "https://open.api.nexon.com/maplestory/v1/ranking/overall"
params = {"date": date, "ocid": ocid}

res = requests.get(url, headers=headers, params=params)
print(res.status_code)
print(res.json())  # 진짜 응답 구조 출력!


200
{'ranking': [{'date': '2024-10-24', 'world_name': '스카니아', 'ranking': 5171, 'character_name': '댜쨈', 'character_level': 290, 'character_exp': 48008130199461, 'class_name': '마법사', 'sub_class_name': '아크메이지(불,독)', 'character_popularity': 64, 'character_guildname': '미담'}]}


In [46]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# ✅ API 설정
API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {"x-nxopen-api-key": API_KEY}
session = requests.Session()
session.headers.update(headers)

# ✅ 입력 데이터
df_input = pd.read_csv("df_missing_ocids_by_week.csv")
input_list = df_input[["ocid", "date"]].dropna().to_dict("records")

# ✅ 상세 정보 수집 함수 (character_name, world_name 포함)
def fetch_details_by_ocid_date(entry):
    ocid = entry["ocid"]
    date = entry["date"]

    result = {
        "ocid": ocid, "date": date,
        "status_code": None,
        "character_name": None, "world_name": None,
        "character_gender": None, "character_class": None, "character_class_level": None,
        "character_level": None, "character_exp": None, "character_exp_rate": None,
        "character_guild_name": None, "character_date_create": None,
        "access_flag": "false", "liberation_quest_clear_flag": "false",
        "popularity": None, "arcane_sum": None, "authentic_sum": None,
        "set_absorlab_count": 0, "set_lubatiss_count": 0, "set_arcane_count": 0, "set_eternal_count": 0,
        "dojang_best_floor": None, "union_level": None,
        "union_artifact_level": None, "union_artifact_exp": None, "union_artifact_point": None
    }

    try:
        res = session.get(f"https://open.api.nexon.com/maplestory/v1/character/basic?ocid={ocid}&date={date}", timeout=10)
        result["status_code"] = res.status_code
        if res.status_code == 200:
            j = res.json()
            result.update({
                "character_name": j.get("character_name"),
                "world_name": j.get("world_name"),
                "character_gender": j.get("character_gender"),
                "character_class": j.get("character_class"),
                "character_class_level": j.get("character_class_level"),
                "character_level": j.get("character_level"),
                "character_exp": j.get("character_exp"),
                "character_exp_rate": j.get("character_exp_rate"),
                "character_guild_name": j.get("character_guild_name"),
                "character_date_create": j.get("character_date_create"),
                "access_flag": j.get("access_flag", "false"),
                "liberation_quest_clear_flag": j.get("liberation_quest_clear_flag", "false")
            })
    except Exception as e:
        print(f"❌ 기본 정보 오류: {ocid} ({date}) - {e}")

    for endpoint, parser in [
        (f"character/popularity", lambda r: {"popularity": r.get("popularity")}),
        (f"character/symbol-equipment", lambda r: {
            "arcane_sum": sum(s.get("symbol_level", 0) for s in r.get("symbol", []) if "아케인" in s.get("symbol_name", "")),
            "authentic_sum": sum(s.get("symbol_level", 0) for s in r.get("symbol", []) if "어센틱" in s.get("symbol_name", ""))
        }),
        (f"character/set-effect", lambda r: {
            "set_absorlab_count": sum(s.get("total_set_count", 0) for s in r.get("set_effect", []) if "앱솔랩스" in s.get("set_name", "")),
            "set_lubatiss_count": sum(s.get("total_set_count", 0) for s in r.get("set_effect", []) if "루타비스" in s.get("set_name", "")),
            "set_arcane_count": sum(s.get("total_set_count", 0) for s in r.get("set_effect", []) if "아케인셰이드" in s.get("set_name", "")),
            "set_eternal_count": sum(s.get("total_set_count", 0) for s in r.get("set_effect", []) if "에테르넬" in s.get("set_name", ""))
        }),
        (f"character/dojang", lambda r: {"dojang_best_floor": r.get("dojang_best_floor")}),
        (f"user/union", lambda r: {
            "union_level": r.get("union_level"),
            "union_artifact_level": r.get("union_artifact_level"),
            "union_artifact_exp": r.get("union_artifact_exp"),
            "union_artifact_point": r.get("union_artifact_point")
        })
    ]:
        try:
            url = f"https://open.api.nexon.com/maplestory/v1/{endpoint}?ocid={ocid}&date={date}"
            res = session.get(url, timeout=10)
            if res.status_code == 200:
                result.update(parser(res.json()))
        except: pass

    print(f"✅ {ocid} ({date}) - 상태코드: {result['status_code']}")
    return result

# ✅ 병렬 실행 + tqdm 진행률 표시
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_details_by_ocid_date, row) for row in input_list]
    for future in tqdm(as_completed(futures), total=len(futures)):
        res = future.result()
        if res: results.append(res)

# ✅ 저장
df_results = pd.DataFrame(results)
df_results.to_csv("recovered_character_details_verbose.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료: recovered_character_details_verbose.csv")


  0%|          | 11/5104 [00:00<02:50, 29.92it/s]

✅ 315bde624cd11939bdbbabd4106fbe4f (2024-10-24) - 상태코드: 200
✅ 9e4feb1cf263a60b9df84e0bb9b3aee7 (2024-10-24) - 상태코드: 200
✅ db0f1d9a578f1497f255d79c6d1abe48 (2024-10-24) - 상태코드: 200
✅ 3327ccbd3400f6e634f96f2a8fcd162defe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ 264d650994bf642095df232f7dc4102f (2024-10-24) - 상태코드: 200
✅ 7405c8fde61d4e4851cd1aa898d8d5b8 (2024-10-24) - 상태코드: 200
✅ a7e13030ee8c6e51786167354a2ce990 (2024-10-24) - 상태코드: 200
✅ 991f8e71da1217400748eed40b8bdf23efe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ 8839da22fb9364c6aabaa31bc982a7c3 (2024-10-24) - 상태코드: 200
✅ 24cbbe59b7b772e1748abfa6a0ced4edefe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ 48e293fc7543f05bc30bf6108b0eefdfefe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ af7df64ff10600cd3f87ebf9f384b6d3efe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ 3d9a94d775603949b0ae473c50a3f4b5a2186d8e7cf7a6ee152d97cc78f2ae8b (2024-10-24) - 상태코드: 200
✅ 807feeccc912b2f985bb4f603a

  1%|          | 30/5104 [00:00<02:04, 40.69it/s]

✅ 3a7e08a69892768f972496b0079ac613 (2024-10-24) - 상태코드: 200
✅ 08d4c15fef4abbb7fafdb0c8e32569cc (2024-10-24) - 상태코드: 200
✅ f8bccbb2b8e11c398cd4def2a4d537e6efe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ 410d83c87d0f64f36d085a2fd5683716 (2024-10-24) - 상태코드: 200
✅ b41af48f5c8f99c0a72b347067009184 (2024-10-24) - 상태코드: 200
✅ 7098187b25417dbd097d65d40b67a4f3efe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ 4dfcb54e90ede3a5282be3d978d2cfe2 (2024-10-24) - 상태코드: 200
✅ bdce92e7d77d572d4fdcc92144dff95c (2024-10-24) - 상태코드: 200
✅ 04397613111392e7101191c8b2ec61b3 (2024-10-24) - 상태코드: 200
✅ 40d94d313b4c225242e3e3bd800f012befe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200
✅ 2b385ec791f74c59a2a7ac45a5956ff1efe8d04e6d233bd35cf2fabdeb93fb0d (2024-10-24) - 상태코드: 200


  1%|          | 40/5104 [00:01<01:53, 44.43it/s]

...

100%|█████████▉| 5090/5104 [01:49<00:00, 49.54it/s]

✅ a65581762b5d6e34bac6c5bf505ab79c (2024-12-19) - 상태코드: 200
✅ 5fae23b0a50206c30cbe3d15b462b589efe8d04e6d233bd35cf2fabdeb93fb0d (2024-12-19) - 상태코드: 200
✅ d89293632bd02624ab8282e83d3dbe0b (2024-12-19) - 상태코드: 200
✅ 514cc55795d14102e6e06877cb5fef6a (2024-12-19) - 상태코드: 200
✅ 2d6bbb92c21e0ae183e7c969df0085a5 (2024-12-19) - 상태코드: 200
✅ 51dabf2f8718e6b757c4dfae215cbbd2 (2024-12-19) - 상태코드: 200
✅ 2d9eafc741722263c64a25aa674f21c4 (2024-12-19) - 상태코드: 200
✅ 96fdb3ef9d3e17227700c25a56e592b2 (2024-12-19) - 상태코드: 200
✅ 9a269c49588c1a45c5b382895206a293 (2024-12-19) - 상태코드: 200
✅ f02dbeb0b14c3143a5ba532ca30fc43fefe8d04e6d233bd35cf2fabdeb93fb0d (2024-12-19) - 상태코드: 200
✅ 330e83a2349a52c860524ad2a853648c (2024-12-19) - 상태코드: 200


100%|██████████| 5104/5104 [01:49<00:00, 46.43it/s]

✅ 84d5cd478dd3c1e583d94a7bf63721b2efe8d04e6d233bd35cf2fabdeb93fb0d (2024-12-19) - 상태코드: 200
✅ af5deee5f0d367ba389960803ab52983 (2024-12-19) - 상태코드: 200
✅ 728fa75237873734e013c5f128872a7b (2024-12-19) - 상태코드: 200
✅ 90da96cdb2989da5837a1e2f0192eff4 (2024-12-19) - 상태코드: 200
✅ 0f2934b74a1e901e3c4c3cb8338740ed (2024-12-19) - 상태코드: 200
✅ d681a590b51294681b60695bde93e2dd (2024-12-19) - 상태코드: 200
✅ 023e363b5eb4997ade40e4f971cc72ad (2024-12-19) - 상태코드: 200
✅ 3b2166fca378cbd08edaf70c9b49b585 (2024-12-19) - 상태코드: 200
✅ 3e856e0b915cb62838958909c60e59d5efe8d04e6d233bd35cf2fabdeb93fb0d (2024-12-19) - 상태코드: 200


✅ 저장 완료: recovered_character_details_verbose.csv


In [47]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# 🔐 API 설정
API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {"x-nxopen-api-key": API_KEY}
session = requests.Session()
session.headers.update(headers)

# 📥 상태코드 400이었던 항목 로드
df_check = pd.read_csv("final_retry_status_400.csv")

# ✅ 결과 저장용 리스트
status_check = []

# ✅ 하나씩 호출해서 존재 여부만 확인
for _, row in tqdm(df_check.iterrows(), total=len(df_check)):
    ocid = row["ocid"]
    date = row["date"]
    status = {
        "ocid": ocid,
        "date": date,
        "status_code": None,
        "result": "FAIL"  # 기본 실패로 시작
    }

    try:
        url = f"https://open.api.nexon.com/maplestory/v1/character/basic?ocid={ocid}&date={date}"
        res = session.get(url, headers=headers, timeout=10)
        status["status_code"] = res.status_code
        if res.status_code == 200:
            status["result"] = "SUCCESS"
    except Exception as e:
        status["result"] = f"EXCEPTION: {e}"

    status_check.append(status)
    time.sleep(0.2)  # 과도한 요청 방지

# ✅ 결과 저장
df_status_check = pd.DataFrame(status_check)
df_status_check.to_csv("checked_ocid_date_status.csv", index=False, encoding="utf-8-sig")
print("✅ 완료! 결과 저장: checked_ocid_date_status.csv")


100%|██████████| 10/10 [00:02<00:00,  3.47it/s]

✅ 완료! 결과 저장: checked_ocid_date_status.csv


In [56]:
import pandas as pd

# 1. 원본과 보강 데이터 불러오기
df_original = pd.read_csv("merged_character_with_exp_access.csv")
df_fill = pd.read_csv("filled_dropped_users_with_nan.csv")

# 2. ocid 기준 병합 (fill된 데이터에서 필요한 컬럼만 병합)
fill_cols = [col for col in df_fill.columns if col != "ocid"]
df_merged = pd.merge(df_original, df_fill, on="ocid", how="left", suffixes=("", "_filled"))

# 3. NaN인 셀에 대해서만 보강 값 채워넣기
for col in fill_cols:
    df_merged[col] = df_merged[col].combine_first(df_merged[f"{col}_filled"])
    df_merged.drop(columns=[f"{col}_filled"], inplace=True)

# 4. 저장 (선택 사항)
df_merged.to_csv("merged_character_with_exp_access_filled.csv", index=False, encoding="utf-8-sig")
print("✅ 병합 및 보강 완료! 파일 저장됨: merged_character_with_exp_access_filled.csv")


C:\Users\qhwhr\AppData\Local\Temp\ipykernel_31340\849538401.py:13: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_merged[col] = df_merged[col].combine_first(df_merged[f"{col}_filled"])
C:\Users\qhwhr\AppData\Local\Temp\ipykernel_31340\849538401.py:13: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_merged[col] = df_merged[col].combine_first(df_merged[f"{col}_filled"])
C:\Users\qhwhr\AppData\Local\Temp\ipykernel_31340\849538401.py:13: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no 

✅ 병합 및 보강 완료! 파일 저장됨: merged_character_with_exp_access_filled.csv


In [73]:
df = pd.read_csv("final_data.csv")

In [74]:
print(df.isnull().sum())

ocid                               0
ranking                            0
character_name                     0
world_name                         0
character_gender                   0
character_class                    0
character_class_level              0
character_level                    0
character_exp                      0
character_exp_rate                 0
character_guild_name           16829
character_date_create              0
access_flag                        0
liberation_quest_clear_flag        0
popularity                         0
arcane_sum                         0
authentic_sum                      0
set_absorlab_count                 0
set_lubatiss_count                 0
set_arcane_count                   0
set_eternal_count                  0
dojang_best_floor                  1
union_level                        0
union_artifact_level               0
union_artifact_exp                 0
union_artifact_point               0
exp_1017                           0
a

In [59]:
import pandas as pd
import os

# 현재 디렉토리에 있는 파일 기준
file_list = os.listdir()

# 날짜 리스트
dates = [
    "2024-10-24", "2024-10-31", "2024-11-07", "2024-11-14",
    "2024-11-21", "2024-11-28", "2024-12-05", "2024-12-12", "2024-12-19"
]

# 출력 폴더
output_dir = "merged_result"
os.makedirs(output_dir, exist_ok=True)

for date in dates:
    base_name = f"character_level_{date}.csv"
    merge_name = f"character_level_{date}(추).csv"
    output_name = f"character_level_{date}_merged.csv"

    if base_name not in file_list or merge_name not in file_list:
        print(f"❌ 파일 누락: {base_name} 또는 {merge_name}")
        continue

    df_base = pd.read_csv(base_name)
    df_merge = pd.read_csv(merge_name)

    # 원본 컬럼만 유지
    keep_cols = [col for col in df_merge.columns if col in df_base.columns or col == "ocid"]
    df_merge = df_merge[keep_cols]

    # 병합
    df_base.set_index("ocid", inplace=True)
    df_merge.set_index("ocid", inplace=True)

    df_base.update(df_merge)
    df_append = df_merge.loc[~df_merge.index.isin(df_base.index)]
    df_base = pd.concat([df_base, df_append])

    df_base.reset_index(inplace=True)
    df_base.to_csv(os.path.join(output_dir, output_name), index=False)

print("✅ 로컬 기준 병합 완료! 폴더: merged_result")


❌ 파일 누락: character_level_2024-10-17.csv 또는 character_level_2024-10-17(추).csv
✅ 로컬 기준 병합 완료! 폴더: merged_result


In [ ]:
import os

print("📂 현재 디렉토리 파일 목록:")
for f in os.listdir():
    print(f)

In [86]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

#  API 설정
API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {"x-nxopen-api-key": API_KEY}
session = requests.Session()
session.headers.update(headers)

#  입력 데이터
df_input = pd.read_csv("final_data.csv")  # ocid 컬럼 있어야 함
input_list = df_input["ocid"].dropna().unique().tolist()

# 2024-10-10 기준 경험치 수집
def fetch_exp(ocid):
    result = {"ocid": ocid, "date": "2024-10-10", "character_exp": None, "status_code": None}
    try:
        res = session.get(
            f"https://open.api.nexon.com/maplestory/v1/character/basic",
            headers=headers,
            params={"ocid": ocid, "date": "2024-10-10"},
            timeout=10
        )
        result["status_code"] = res.status_code
        if res.status_code == 200:
            result["character_exp"] = res.json().get("character_exp")
    except Exception as e:
        print(f"❌ 오류 발생: {ocid} - {e}")
    return result

# 병렬 처리 + 진행률 표시
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_exp, ocid) for ocid in input_list]
    for future in tqdm(as_completed(futures), total=len(futures)):
        res = future.result()
        if res: results.append(res)

# 저장
df_result = pd.DataFrame(results)
df_result.to_csv("character_exp_20241010.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료: character_exp_20241010.csv")


100%|██████████| 222937/222937 [1:03:29<00:00, 58.52it/s]


✅ 저장 완료: character_exp_20241010.csv


In [2]:
import pandas as pd
df2 = pd.read_csv("character_exp_20241010.csv")

In [7]:
import pandas as pd
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
API_KEY = "live_6da477cce0d1ca918d4917fc55b8268d0849d7b5746a5e7acdace78398868480efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {"x-nxopen-api-key": API_KEY}

# ocid 목록 (NaN 경험치였던 유저들)
df = pd.read_csv("exp_1010_null_rows.csv")  # 또는 df[df["character_exp"].isna()]
ocid_list = df["ocid"].dropna().unique().tolist()
target_date = "2024-10-10"
# 결과 저장 리스트
results = []

# 호출 함수 정의
def fetch_character_exp(ocid):
    url = f"https://open.api.nexon.com/maplestory/v1/character/basic?ocid={ocid}&date={target_date}"
    try:
        res = requests.get(url, headers=headers, timeout=5)
        if res.status_code == 200:
            data = res.json()
            return {
                "ocid": ocid,
                "date": target_date,
                "character_exp": data.get("character_exp"),
                "status_code": res.status_code
            }
        else:
            return {
                "ocid": ocid,
                "date": target_date,
                "character_exp": None,
                "status_code": res.status_code
            }
    except Exception as e:
        return {
            "ocid": ocid,
            "date": target_date,
            "character_exp": None,
            "status_code": str(e)
        }

# 병렬 처리 실행
with ThreadPoolExecutor(max_workers=5) as executor:  # API 초당 5건 제한
    futures = [executor.submit(fetch_character_exp, ocid) for ocid in ocid_list]
    for f in tqdm(as_completed(futures), total=len(futures), desc="🔄 경험치 조회 중"):
        result = f.result()
        results.append(result)
        print(f"[{result['ocid']}] status: {result['status_code']} exp: {result['character_exp']}")

# 저장
df_result = pd.DataFrame(results)
df_result.to_csv("character_exp_20241010_checked_parallel.csv", index=False)


🔄 경험치 조회 중:   3%|▎         | 19/651 [00:00<00:07, 82.32it/s]

[2e6b3002de09e59057e0e3f3c28cd85a] status: 200 exp: None
[744c77c2e5ebefe222a9b274b22740c1efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[8d6f8b21fde141b8a5070fe76077cbf33663933da92e7566c55104ec9ac2894d] status: 200 exp: None
[9a53f62950e4665183f33a463e72949c] status: 200 exp: None
[67e10237952a3ec20ffc6e7e29089191] status: 200 exp: None
[8cb69985ed98c59664d82ca4b4453bcc] status: 200 exp: None
[c90a73157d786b94c12cfcfb39f04292] status: 200 exp: None
[40b6fbdd90742dbcd0eb49ec2717dea8efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[44cb2015349510f3f0a9c6e3cf70a6c8] status: 200 exp: None
[915f518ab6461e9c011fc55e55cc15c2efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[d69a3486f22b67c44afb68e6660aa4b0efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[9fc1c36560ee95ca7bf16597d30a788a] status: 400 exp: None
[31f237ceb50bb1f8dacf1a4abe0e9b38] status: 200 exp: None
[d71612889cb8ba7ee9a9fdad1242042defe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[bc7d7ddde

🔄 경험치 조회 중:   6%|▌         | 39/651 [00:00<00:06, 93.34it/s]

[490fa251f5a6defa6e72772e1b4ccfc0] status: 200 exp: None
[f19fd640eeba51c8a149f96088c750b9efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[c7a927acb726623323b3cd4ca567935defe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[06fa9c3040bd4d82d82164ec5cb9f0b6] status: 200 exp: None
[15bcf6f353cae676699c81acd22a220befe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[bd1dce8e26ab5a5c3e003d69b6b76f0c] status: 200 exp: None
[890dd874b48428f35c69f71138c83ed7] status: 200 exp: None
[9065a752fb396e93fda12fff524e7106] status: 200 exp: None
[a9e25ddb2e45c1bfb92ef3f616552dbb] status: 200 exp: None
[b18d9efe53fab4ed7d047e177692b1a3] status: 200 exp: None
[ff077ac4a8de4c19781d3e039ee647eaefe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[7b0c2b3bb37574aa436bc6c8997bc681] status: 200 exp: None
[2f3f63bd6b197c39bcd6db954e443d6c8295750b1f4857867454debc45ec9c8a] status: 200 exp: None
[7e6ce044b998b76ed5ac984050299351efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[744fcd0d6

🔄 경험치 조회 중:   8%|▊         | 49/651 [00:00<00:06, 93.39it/s]

...

[967fe723c2df7367fe5cad6f2b316985] status: 200 exp: None
[02944b65b633580ba6853b362be13df9] status: 200 exp: None
[98fa394d95ccbc857d023930b6f151d9] status: 200 exp: None
[76af8e252220d1851b8a84f4b4163f52] status: 200 exp: None
[e36fae2cbe6647fe1fe9813616f650f2] status: 200 exp: None
[16e2569061218a20d8064315f724d3ecefe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[9ae0ce6fe16518350a224d6039f3aca8efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[01c7dda3193c63b3a697bcdf40e724ce] status: 200 exp: None
[2c7355e7e74c909c67ae423e8433f16befe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[caa5b2b50ea397e33f856765d9d85cf6] status: 200 exp: None
[7a407cf0a9442e3d479ab7ec8b1fa10b] status: 200 exp: None
[d9e389f8540117b0b20a8200de87f6ae] status: 200 exp: None
[502273dd96c7587624619c4ec6cf5f80efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[bdda848edd65fb2d2ee18d3aa6c14ab0] status: 200 exp: None
[3cec61013b749bbf8958971a62f08381fd842dab14be2a956555625b37c97bff] status:

🔄 경험치 조회 중:  99%|█████████▉| 643/651 [00:06<00:00, 81.16it/s]

[d6e066669e2f2b6ad4d8b96e0f78a07c] status: 200 exp: None
[df0e082a1fcd341ddde0677dc82f005a86e68d4012202d0434d05f81868f68e3] status: 200 exp: None
[e9d8037a8d6bc61b6b421a652ae001e1efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[4357f7c196bf94bd6c9db7f1a115bd8e8295750b1f4857867454debc45ec9c8a] status: 200 exp: None
[3c7ac994fb8e3cb0ca8dc9bb373cb2ab8295750b1f4857867454debc45ec9c8a] status: 200 exp: None
[664fe4f14c09ba79a24919b11c65092f] status: 200 exp: None
[4207fa1533dffdcd10a95d10061a9bd2] status: 200 exp: None
[6f926a6c39a204a4c35862b94183d4f6efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[b98ac4f02f16fbadc149e4667ef6f094efe8d04e6d233bd35cf2fabdeb93fb0d] status: 200 exp: None
[6f9972944906407e76005a5adc0b704cfd842dab14be2a956555625b37c97bff] status: 200 exp: None
[7c1e7f507da23afa47027b4d83c6bbaa] status: 200 exp: None


🔄 경험치 조회 중: 100%|██████████| 651/651 [00:06<00:00, 96.85it/s]

[62a76fdd3e735c27453a3bbca1d76a62] status: 200 exp: None


In [3]:
print(df2.isnull().sum())

ocid               0
date               0
character_exp    651
status_code        0
dtype: int64


In [89]:
import pandas as pd

# 1. 파일 불러오기
df_1010 = pd.read_csv("character_exp_20241010.csv")  # ocid, character_exp
df_final = pd.read_csv("final_data.csv")  # ocid + character_exp_1017 or 유사 컬럼

# 2. 컬럼명 정리
df_1010 = df_1010.rename(columns={"character_exp": "exp_1010"})

# 3. 1017쪽 경험치 컬럼 자동 탐색
exp_col_1017 = None
for col in df_final.columns:
    if "exp" in col and "1017" in col:
        exp_col_1017 = col
        break
if exp_col_1017 is None:
    exp_col_1017 = [col for col in df_final.columns if "exp" in col][0]

df_final = df_final.rename(columns={exp_col_1017: "exp_1017"})

# 4. 병합
df_merged = pd.merge(df_1010[["ocid", "exp_1010"]], df_final, on="ocid", how="inner")

# 5. 경험치 변화 여부
df_merged["exp_changed"] = df_merged["exp_1010"] != df_merged["exp_1017"]

# 6. 분리
df_changed = df_merged[df_merged["exp_changed"] == True].drop(columns=["exp_changed"])
df_unchanged = df_merged[df_merged["exp_changed"] == False].drop(columns=["exp_changed"])

# 7. 저장
df_changed.to_csv("final_data_changed_only.csv", index=False, encoding="utf-8-sig")
df_unchanged.to_csv("final_data_unchanged_only_with_exp1010.csv", index=False, encoding="utf-8-sig")

print("✅ 저장 완료:")
print("- 변화 있음: final_data_changed_only.csv")
print("- 변화 없음 (exp_1010 포함): final_data_unchanged_only_with_exp1010.csv")


✅ 저장 완료:
- 변화 있음: final_data_changed_only.csv
- 변화 없음 (exp_1010 포함): final_data_unchanged_only_with_exp1010.csv
